In [1]:
!pip install qiskit[nature]
!pip install numpy
!pip install matplotlib 
!pip install pyscf

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 30.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 36.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 241.3/241.3 KB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 55.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.0/107.0 KB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 KB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.9/55.9 KB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 KB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.5/37.5 MB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 K

In [38]:
from qiskit_nature.units import DistanceUnit
from qiskit_nature.second_q.drivers import PySCFDriver

driver = PySCFDriver(
    atom="Be 0 0 0; H 0 0 1.3; H 0 0 -1.3",
    basis="sto3g",
    charge=0,
    spin=0,
    unit=DistanceUnit.ANGSTROM,
)

es_problem = driver.run()
full_problem = driver.run()
fermionic_op = full_problem.hamiltonian.second_q_op()
print(full_problem.molecule)
print(full_problem.num_particles)
print(full_problem.num_spatial_orbitals)

Molecule:
	Multiplicity: 1
	Charge: 0
	Unit: Bohr
	Geometry:
		Be	(0.0, 0.0, 0.0)
		H	(0.0, 0.0, 2.4566439619345806)
		H	(0.0, 0.0, -2.4566439619345806)
	Masses:
		Be	9
		H	1
		H	1
(3, 3)
7


In [39]:
from qiskit_nature.second_q.transformers import FreezeCoreTransformer

fc_transformer = FreezeCoreTransformer()

fc_problem = fc_transformer.transform(full_problem)
print(fc_problem.num_particles)
print(fc_problem.num_spatial_orbitals)

(2, 2)
6


In [40]:
print(fc_problem.hamiltonian.constants)


{'nuclear_repulsion_energy': 3.460004840630769, 'FreezeCoreTransformer': -15.067622237673163}


In [42]:
from qiskit_nature.second_q.mappers import JordanWignerMapper, QubitConverter, ParityMapper

converter = QubitConverter(ParityMapper(), two_qubit_reduction=True, z2symmetry_reduction="auto")
qubit_op = converter.convert(fermionic_op, num_particles=full_problem.num_particles, sector_locator=full_problem.symmetry_sector_locator)
print(qubit_op)

-11.887525109778208 * IIIIIIIII
+ 2.2173551483181826 * IIIIIIIIZ
+ 0.032518479512300615 * ZZIIZIZZX
+ 0.032518479512300615 * IIIIIIIIX
+ 0.024948945341574772 * IIIIIIZXX
+ 0.024948945341574772 * IIIIIIIYY
- 0.008801797169126538 * ZZIIZIZZZ
- 0.11636082853291636 * ZZIIZIZZI
- 0.028467913806896204 * IIIIIIZXZ
- 0.028467913806896217 * ZZIIZIZXI
- 0.020569420820584702 * IIIIIIZXI
- 0.020569420820584702 * ZZIIZIZXZ
+ 0.021297363813995922 * ZIIZZIZII
- 0.11440065891301435 * ZIIZZIZIZ
- 0.010992106728516393 * ZZIIZIYYI
+ 0.010992106728516365 * IZIZIIYYI
- 0.03161788498925638 * ZZIIZIYYZ
+ 0.03161788498925638 * IZIZIIYYZ
- 0.2937994092312812 * IIZZIIIII
+ 0.28064386882572134 * IIZZIIIIZ
- 0.2937994092312823 * IZZIIIIII
+ 0.2806438688257213 * IZZIIIIIZ
- 0.271391541742038 * IIIIIIZZI
+ 0.11363715165651848 * IIIIIIZZZ
+ 0.49259598879097855 * IIIIIIZII
- 0.13995225430993327 * IIIIIIZIZ
+ 4.010823393073848e-05 * ZZIIZIIYY
+ 4.010823393073848e-05 * ZZIIZIZXX
- 0.004400676695225225 * IZIZIIIZX
- 0.0

In [43]:
from qiskit.algorithms.minimum_eigensolvers import NumPyMinimumEigensolver

numpy_solver = NumPyMinimumEigensolver()


In [44]:
from qiskit.algorithms.optimizers import SLSQP
from qiskit.primitives import Estimator
from qiskit_nature.second_q.algorithms import VQEUCCFactory
from qiskit_nature.second_q.circuit.library import UCCSD

vqe_solver = VQEUCCFactory(Estimator(), UCCSD(), SLSQP())

from qiskit_nature.second_q.algorithms import GroundStateEigensolver

calc = GroundStateEigensolver(converter, vqe_solver)

In [45]:

res = calc.solve(es_problem)
print(res)

=== GROUND STATE ENERGY ===
 
* Electronic ground state energy (Hartree): -19.054689467498
  - computed part:      -19.054689467498
~ Nuclear repulsion energy (Hartree): 3.460004840631
> Total ground state energy (Hartree): -15.594684626867
 
=== MEASURED OBSERVABLES ===
 
  0:  # Particles: 6.000 S: 0.000 S^2: 0.000 M: -0.000
 
=== DIPOLE MOMENTS ===
 
~ Nuclear dipole moment (a.u.): [0.0  0.0  0.0]
 
  0: 
  * Electronic dipole moment (a.u.): [None  None  None]
    - computed part:      [None  None  None]
  > Dipole moment (a.u.): [None  None  None]  Total: None
                 (debye): [None  None  None]  Total: None
 


In [46]:
calc = GroundStateEigensolver(converter, numpy_solver)
res = calc.solve(es_problem)
print(res)

=== GROUND STATE ENERGY ===
 
* Electronic ground state energy (Hartree): -19.055051921565
  - computed part:      -19.055051921565
~ Nuclear repulsion energy (Hartree): 3.460004840631
> Total ground state energy (Hartree): -15.595047080935
 
=== MEASURED OBSERVABLES ===
 
  0:  # Particles: 6.000 S: 0.000 S^2: 0.000 M: 0.000
 
=== DIPOLE MOMENTS ===
 
~ Nuclear dipole moment (a.u.): [0.0  0.0  0.0]
 
  0: 
  * Electronic dipole moment (a.u.): [None  None  None]
    - computed part:      [None  None  None]
  > Dipole moment (a.u.): [None  None  None]  Total: None
                 (debye): [None  None  None]  Total: None
 
